In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
from scipy.io import loadmat
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from matplotlib.text import Text

In [2]:
path = os.path.abspath('../../1_Simulation_Results/numpy_files/')

colormap_data = loadmat('../cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

In [3]:
model_GaussianTDB = np.load(path+'/Gaussian/TDB/Gaussian_TDB.npz')

LIQ_GT     = model_GaussianTDB['arr1']
FCC_GT     = model_GaussianTDB['arr2']
vel_GT     = model_GaussianTDB['arr3']
temp_GT    = model_GaussianTDB['arr4']
timeS_GT  = model_GaussianTDB['arr5']

time_steps_GT =  timeS_GT.shape[0]

## Phase

In [4]:
def update_plot(t_step):
    scan_speed = r'30 $\mathbf{\mu m/s}$'
    power = r"25 $\mathbf{mW}$"
    laser_speed = 30
    scaling_factor = 401/1000 # scaling factor for plotting figure
    laser_act_pos = 125 + timeS_GT[t_step]*laser_speed  # Laser actual position in true dimension
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    cmap = plt.get_cmap('RdYlGn_r')
    cmap.set_under('white', alpha=0)
    hmap1 = ax1.imshow(LIQ_GT[:,:,t_step], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
    
    cmap = plt.get_cmap('Wistia')
    cmap.set_under('white', alpha=0) 
    hmap2 = ax1.imshow(FCC_GT[:,:,t_step], cmap=cmap, vmin=0.5, vmax=1.5, aspect=0.5, interpolation='quadric')
    
    contour_level_LIQ = [0.8]
    contour_LIQ = plt.contour(LIQ_GT[:, :, t_step], levels=contour_level_LIQ, colors='k', linestyles='--', linewidths=3,)
    
    contour_levels = [450, 700,  1337]
    label_colors = ['k', 'k', 'aqua']
    contour = ax1.contour(temp_GT[:, :, t_step], levels=contour_levels, colors='crimson', linewidths=3, linestyles='dashdot')
    [(label.set_fontweight('bold'), label.set_color(color)) for label, color in zip(plt.clabel(contour, inline=True, fontsize=24, fmt='%1.0f K'), label_colors)]
        
    if timeS_GT[t_step] <= 25:
        ax1.arrow(laser_pos, -80, 0, 76,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-15,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
        if laser_act_pos < 500:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')
            ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        else:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
            ax1.text(laser_pos-65, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos-65, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
    else:
        ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
        ax1.text(350, -40, "LASER", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.text(350, -20, "OFF", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.arrow(400, -10, -50, 0, fill=False, hatch='x', overhang=1, linestyle='--',  width = 3, color='blue', alpha=1, length_includes_head=True, clip_on=False)
        ax1.arrow(350, -80, 0, 76,  width = 4.5, color='none', length_includes_head=True, clip_on=False) # to make the figure height same after laser is turned off
    
    AuFCC_txt = ax1.text(20,180,'FCC', fontsize = 40, weight='bold', color='white')
    if timeS_GT[t_step] <= 25:
        AuLIQ_txt = ax1.text(laser_pos-20, 30,'LIQ_GT', fontsize = 35, weight='bold', color='white')
    
    ax1.set_ylim(201, -1)    
    ax1.set_xlim(-1,401)
    
    ################## Dimensions ###################
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 1000 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$ 250 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(4)


t_steps = 523
############ To save the image as well #############
# for t_step in range(300, 523, 1):
for t_step in range(t_steps):
    update_plot(t_step)
    plt.savefig(f'../temperoray_figures/Gaussian/TDB/phase/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close() 
print("FINISHED")

FINISHED


## Temperature

In [5]:
def update_plot(t_step):
    scan_speed = r'30 $\mathbf{\mu m/s}$'
    power = r"25 $\mathbf{mW}$"
    laser_speed = 30
    scaling_factor = 401/1000 # scaling factor for plotting figure
    laser_act_pos = 125 + timeS_GT[t_step]*laser_speed  # Laser actual position in true dimension
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    hmap1 = ax1.imshow(temp_GT[:,:,t_step], cmap='RdYlBu_r', vmin=300, aspect=0.5,  interpolation='quadric')
    
    if timeS_GT[t_step] <= 25:
        ax1.arrow(laser_pos, -80, 0, 76,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-15,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
        if laser_act_pos < 500:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')
            ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        else:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
            ax1.text(laser_pos-65, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos-65, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
    else:
        ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
        ax1.text(350, -40, "LASER", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.text(350, -20, "OFF", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.arrow(400, -10, -50, 0, fill=False, hatch='x', overhang=1, linestyle='--',  width = 3, color='blue', alpha=1, length_includes_head=True, clip_on=False)
        ax1.arrow(350, -80, 0, 76,  width = 4.5, color='none', length_includes_head=True, clip_on=False) # to make the figure height same after laser is turned off
    
    ax1.set_ylim(201, -1)    
    ax1.set_xlim(-1,401)
    
    contour_levels = [450, 700,  1337]
    label_colors = ['tomato', 'orangered', 'k']
    contour = plt.contour(temp_GT[:, :, t_step], levels=contour_levels, colors='black', linewidths=4, linestyles='dotted')
    [(label.set_fontweight('bold'), label.set_color(color)) for label, color in zip(plt.clabel(contour, inline=True, fontsize=24, fmt='%1.0f K'), label_colors)]
    
    ################## Dimensions ###################
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 1000 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$ 250 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ############## Color Bar ###########
    ax2 = fig.add_axes([0.92, 0.3, 0.05, 0.39]) 
    ax2.tick_params(axis='both', labelcolor='black', labelsize=1, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    cbar = fig.colorbar(hmap1, cax=ax2, orientation='vertical')
    cbar.ax.tick_params(labelsize=25, direction='in', length=0) 
    # cbar.set_label(r'$\mathbf{\leftarrow}$   T [K]  $\mathbf{\rightarrow}$', color='k',  labelpad=-115, fontsize=30, rotation=90, weight='bold')
    text = Text(x=0.935, y=0.335, text=r'$\mathbf{\leftarrow}$   T [K]  $\mathbf{\rightarrow}$', fontsize=30, color='k', rotation=90, weight='bold')
    fig.add_artist(text)
    if timeS_GT[t_step] <= 25:
        cbar.set_ticks([np.min(temp_GT[:,:,t_step])*1.06, (np.max(temp_GT[:,:,t_step]))/2, np.max(temp_GT[:,:,t_step])*0.96], labels=[f'  {np.min(temp_GT[:,:,t_step]):.0f}', f'{np.max(temp_GT[:,:,t_step])/2 :.0f}', f'{np.max(temp_GT[:,:,t_step])*0.95:.0f}'], weight='bold') 
    else:
        cbar.set_ticks([np.min(temp_GT[:,:,t_step]), (np.max(temp_GT[:,:,t_step])+np.min(temp_GT[:,:,t_step]))/2, np.max(temp_GT[:,:,t_step])], labels=[f'  {np.min(temp_GT[:,:,t_step]):.0f}', f'  {(np.max(temp_GT[:,:,t_step])+np.min(temp_GT[:,:,t_step]))/2 :.0f}', f'  {np.max(temp_GT[:,:,t_step]):.0f}'], weight='bold') 
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(4)
    ax1.spines[:].set_color('k')
    ax2.spines[:].set_linewidth(4)
    line = Line2D([1.04, 1.04], [0.35, 0.65], transform=fig.transFigure, color="none", linewidth=3, linestyle='-')
    fig.add_artist(line)

t_steps = 520
############ To save the image as well #############
# for t_step in range(500, 520, 1):
for t_step in range(t_steps):
    update_plot(t_step)
    plt.savefig(f'../temperoray_figures/Gaussian/TDB/temperature/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close() 
print("FINISHED")

FINISHED


## Velocity

In [6]:
def update_plot(t_step):
    scan_speed = r'30 $\mathbf{\mu m/s}$'
    power = r"25 $\mathbf{mW}$"
    laser_speed = 30
    scaling_factor = 401/1000 # scaling factor for plotting figure
    laser_act_pos = 125 + timeS_GT[t_step]*laser_speed  # Laser actual position in true dimension
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    hmap1 = ax1.imshow(vel_GT[:,:,t_step], cmap='gist_ncar_r', vmin=2, aspect=0.5,  interpolation='bilinear')
    
    ######################### Laser Arrow ###########################
    if timeS_GT[t_step] <= 25:
        ax1.arrow(laser_pos, -80, 0, 76,  width = 4.5, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-15,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
        if laser_act_pos < 500:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')
            ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        else:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
            ax1.text(laser_pos-65, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos-65, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
    else:
        ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
        ax1.text(350, -40, "LASER", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.text(350, -20, "OFF", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.arrow(400, -10, -50, 0, fill=False, hatch='x', overhang=1, linestyle='--',  width = 3, color='blue', alpha=1, length_includes_head=True, clip_on=False)
        ax1.arrow(350, -80, 0, 76,  width = 4.5, color='none', length_includes_head=True, clip_on=False) # to make the figure height same after laser is turned off
        
    contour_level_LIQ = [0.8]
    contour_LIQ = plt.contour(LIQ_GT[:, :, t_step], levels=contour_level_LIQ, colors='orangered', linestyles='-', linewidths=5,)
    
    contour_levels = [450, 700, 1337]
    contour = plt.contour(temp_GT[:, :, t_step], levels=contour_levels, colors='lime', linewidths=2,)
    clabels = plt.clabel(contour, inline=True, fontsize=24, colors='k', fmt='%1.0f K')
    
    ax1.set_ylim(201, -1)    
    ax1.set_xlim(-1,401)
    
    for label in clabels: label.set_fontweight('bold')
        
    ################## Dimensions ###################
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 1000 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$ 250 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ############## Color Bar ###########
    ax2 = fig.add_axes([0.92, 0.3, 0.05, 0.39]) 
    ax2.tick_params(axis='both', labelcolor='black', labelsize=1, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    cbar = fig.colorbar(hmap1, cax=ax2, orientation='vertical')
    cbar.ax.tick_params(labelsize=25, direction='in', length=0) 
    cbar.set_label(r'$\mathbf{\leftarrow}$   Vel $\mathbf{[\mu m/s]}$  $\mathbf{\rightarrow}$', color='k',  labelpad=20, fontsize=20, rotation=90, weight='bold')
    cbar.set_ticks([np.max(vel_GT[:,:,t_step])*0.06, (np.max(vel_GT[:,:,t_step]))/2, np.max(vel_GT[:,:,t_step])*0.96], labels=[f'{np.min(vel_GT[:,:,t_step]):.0f}', f'{np.max(vel_GT[:,:,t_step])/2 :.0f}', f'{np.max(vel_GT[:,:,t_step]):.0f}'], weight='bold') 
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(4)
    ax1.spines[:].set_color('b')
    ax2.spines[:].set_linewidth(4)
    line = Line2D([1.041, 1.041], [0.35, 0.65], transform=fig.transFigure, color="none", linewidth=3, linestyle='-')
    fig.add_artist(line)

t_steps = 437
############ To save the image as well #############
# for t_step in range(100, 437, 1):
for t_step in range(t_steps):
    update_plot(t_step)
    plt.savefig(f'../temperoray_figures/Gaussian/TDB/velocity/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close() 
print("FINISHED")

FINISHED


## Lead_Lag

In [13]:
import warnings
warnings.filterwarnings("ignore", message="No artists with labels found to put in legend")

In [4]:
end_step = max((i for i, v in enumerate(list(timeS_GT)) if v <= 25), key=lambda i: list(timeS_GT)[i])
end_step

435

In [5]:
def leading_front(array, condition, end_time_step):
    condition = condition
    end_step = end_time_step
    leading_front = []
    for t in range(end_step):
        A = array[0:1,:,t][0]
        for i in range(len(A)-1,0,-1):
            if A[i]>=condition:
                # print(i)
                break
        leading_front.append(i)
    return leading_front

In [6]:
leading_temperature = leading_front(array=temp_GT[:,:,:end_step], condition=1337, end_time_step=end_step)
leading_LIQ = leading_front(array=LIQ_GT[:,:,:end_step], condition=0.5, end_time_step= end_step)
time_duration = np.arange(end_step)
time_lead_lag =  timeS_GT[:end_step]

In [7]:
def fit_real_time(data, order):
    position, time_lead_lag = data
    coefficients_real = np.polyfit(position, time_lead_lag, order)
    fitted_real_time = np.poly1d(coefficients_real)
    return fitted_real_time
    
common_position = leading_LIQ.index(leading_temperature[1]) # = 36 ## => comes from nearest value of starting point of leading temp_GT i.e. => leading_temperature[1]
phase_fit_real_time = fit_real_time([leading_LIQ[1:], time_lead_lag[1:]], 15)
temp_fit_real_time = fit_real_time([leading_temperature[1:], time_lead_lag[1:]], 15)

/tmp/ipykernel_46150/2066266000.py:3: RankWarning: Polyfit may be poorly conditioned
  coefficients_real = np.polyfit(position, time_lead_lag, order)


In [8]:
lag_time = []
for t in range(end_step):
    lag_time.append(phase_fit_real_time(leading_LIQ[t]) - temp_fit_real_time(leading_LIQ[t]))

#### loop

In [ ]:
def update_plot(t_step):
    scan_speed = r'30 $\mathbf{\mu m/s}$'
    power = r"25 $\mathbf{mW}$"
    
    laser_speed = 30
    scaling_factor = 401/1000 # scaling factor for plotting figure
    laser_act_pos = 125 + timeS_GT[t_step]*laser_speed  # Laser actual position in true dimension
    laser_pos = laser_act_pos*scaling_factor # Laser position in accordance with figure
    
    x = np.arange(time_steps_GT-1)
    
    fig, ax1 = plt.subplots(1,1, figsize=(16,8), frameon=False)
    ax1.set_facecolor((0, 0, 0, 0))
    
    hmap1 = ax1.imshow(temp_GT[:,:,t_step], cmap='Spectral_r', vmin=300, aspect=0.5,  interpolation='quadric')
    
    ax1.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    ax1.spines[:].set_linewidth(4)
    ax1.spines[:].set_color('k')
    
    if timeS_GT[t_step] <= 25:
        ax1.arrow(laser_pos,-75, 0, 71,  width = 4, color='red', length_includes_head=True, clip_on=False)
        ax1.text(laser_pos-15,-12, power, rotation=90, color='k', fontsize=20, weight='bold')
        if laser_act_pos < 500:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')
            ax1.text(laser_pos+10, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos+10, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
        else:
            ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
            ax1.text(laser_pos-55, -40, s=f'{scan_speed}', fontsize=20, weight = 'bold')
            ax1.arrow(laser_pos-55, -28, 30, 0,  width = 3, color='k', fill=False, clip_on=False)
    else:
        ax1.set_title(f't = {timeS_GT[t_step]:.2f}s', pad=10, loc='left', fontsize=25, weight='bold')
        ax1.text(350, -40, "LASER", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.text(350, -20, "OFF", rotation=0, color='k', alpha=1, fontsize=20, weight='bold')
        ax1.arrow(400, -10, -50, 0, fill=False, hatch='x', overhang=1, linestyle='--',  width = 3, color='blue', alpha=1, length_includes_head=True, clip_on=False)
        ax1.arrow(350, -75, 0, 71,  width = 4, color='none', length_includes_head=True, clip_on=False) # to make the figure height same after laser is turned off
    
    
    ################## Meltpool and Isotherm ################
    cmap = plt.get_cmap('Wistia')
    cmap.set_under('white', alpha=0) 
    meltpool_1 = ax1.imshow(LIQ_GT[:,:,t_step], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
    ax1.arrow(leading_temperature[t_step], -14, 0, 10,  width = 7, head_width=7, color='lime', length_includes_head=True, clip_on=False)
    ax1.text(leading_temperature[t_step]+3, -8, s=r'$\tau$', fontsize=25, weight = 'bold')
    ax1.arrow(leading_LIQ[t_step], 11, 0, -10,  width = 7, head_width=7,  color='b', length_includes_head=True, clip_on=False)
    ax1.text(leading_LIQ[t_step]+5, 20, s=r'$\mathfrak{D}$', fontsize=25, weight = 'bold')
    
    ################# Contour ##################
    contour_levels = [1337]
    contour = plt.contour(temp_GT[:, :, t_step], levels=contour_levels, colors='orangered', linewidths=4,)
    clabels = plt.clabel(contour, inline=True, fontsize=15, colors='k', fmt='%1.0f K')
    for label in clabels: label.set_fontweight('bold')
    
    ############## Color Bar ###########
    ax2 = fig.add_axes([0.92, 0.3, 0.05, 0.39]) 
    ax2.tick_params(axis='both', labelcolor='black', labelsize=1, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    cbar = fig.colorbar(hmap1, cax=ax2, orientation='vertical')
    cbar.ax.tick_params(labelsize=25, direction='in', length=0) 
    text = Text(x=0.935, y=0.335, text=r'$\mathbf{\leftarrow}$   T [K]  $\mathbf{\rightarrow}$', fontsize=30, color='k', rotation=90, weight='bold')
    fig.add_artist(text)
    ax2.spines[:].set_linewidth(4)
    
    if timeS_GT[t_step] <= 25:
        cbar.set_ticks([np.min(temp_GT[:,:,t_step])*1.06, (np.max(temp_GT[:,:,t_step]))/2, np.max(temp_GT[:,:,t_step])*0.96], labels=[f'{np.min(temp_GT[:,:,t_step]):.0f}', f'{np.max(temp_GT[:,:,t_step])/2 :.0f}', f'{np.max(temp_GT[:,:,t_step])*0.95:.0f}'], weight='bold') 
    else:
        cbar.set_ticks([np.min(temp_GT[:,:,t_step]), (np.max(temp_GT[:,:,t_step])+np.min(temp_GT[:,:,t_step]))/2, np.max(temp_GT[:,:,t_step])], labels=[f'{np.min(temp_GT[:,:,t_step]):.0f}', f'{(np.max(temp_GT[:,:,t_step])+np.min(temp_GT[:,:,t_step]))/2 :.0f}', f'{np.max(temp_GT[:,:,t_step]):.0f}'], weight='bold') 
    
    ################## Dimensions ###################
    ax1.set_xlabel('$\mathbf{\longleftarrow}$                 1000 $ \mathbf{\mu m}$                $\mathbf{\longrightarrow}$', fontsize = 27, weight ='bold',)
    ax1.annotate(r'$\mathbf{\leftarrow}$ 250 $\mathbf{\mu m}$ $\mathbf{\rightarrow}$', xy=(0.5, 0.5), xytext=(-0.025, 0.5), rotation=90,
                 xycoords='axes fraction', textcoords='axes fraction', fontsize = 27, weight = 'bold', color='k', ha='center', va='center')
    
    ax1.set_ylim(201, -1)
    ax1.set_xlim(-1,401)
    
    ##############################################
    ##############################################
    if laser_act_pos > 500:
        ax3 = fig.add_axes([0.18, 0.34, 0.2, 0.18], frameon=True)  # [left, bottom, width, height]
    
    else:
        ax3 = fig.add_axes([0.63, 0.34, 0.2, 0.18], frameon=True)  # [left, bottom, width, height]
    # ax3.set_facecolor((0, 0, 0, 0))
    ax3.plot(leading_temperature[1:t_step], time_lead_lag[1:t_step], marker='o', c='lime',markersize=1, linewidth=2, label =r'$\tau$')
    ax3.plot(leading_LIQ[1:t_step], time_lead_lag[1:t_step], marker='.', c='blue',markersize=1,linewidth=2, label =r'$\mathfrak{D}$')
    ax3.set_ylabel("t [s]",  weight = 'bold', size=20, color='blue',labelpad=-1 )
    ax3.set_xticks([55, 200, 380], labels=[ r'X$\mathbf{[\mu m]}\rightarrow$', f'{200*(1000/400):.0f}', f'1000'], size=15, color='white', weight='bold',)
    ax3.xaxis.set_tick_params(direction='in', length=0) 
    ax3.yaxis.set_tick_params(labelcolor='lime', labelsize=15,  direction='inout', length=8, width=2, color='k',)
    for label in ax3.get_yticklabels(): label.set_fontweight('bold')
    ax3.yaxis.set_major_locator(plt.MaxNLocator(4))
    legend_1 = ax3.legend(loc=(0.41,0.57), ncol=2, columnspacing=0.5, prop={'weight':'bold', 'size':'30',}, handletextpad=0.1, handlelength=0.7,   frameon=False).get_frame().set_facecolor('none')
    
    ax4 = ax3.twinx()
    if t_step>common_position:
        ax4.plot(leading_LIQ[common_position:t_step],lag_time[common_position:t_step], linestyle='-',  c='red', linewidth=3, label =r'$\Delta t_{x}$') # positional_timelag
    legend_2 = ax4.legend(loc=(0.74,-0.05),  prop={'weight':'bold', 'size':'20',}, handletextpad=0.1, handlelength=0.75,   frameon=False).get_frame().set_facecolor('none')
    ax4.set_ylabel(r"$\mathbf{\delta t_{pos}}$ [s]",  weight = 'bold', size=20, color='red')
    ax4.yaxis.set_tick_params(labelcolor='red', labelsize=17, direction='inout', length=8, width=2, color='k',)
    ax4.yaxis.set_major_locator(plt.MaxNLocator(4))
    for label in ax4.get_yticklabels(): label.set_fontweight('bold')
    ax4.set_ylim(1.25,1.75)
    
    ax3.spines[:].set_linewidth(2)
    ax3.set_ylim([-2,28])
    ax3.set_xlim([0,420])

    # plt.show()
   
t_steps = 434
############ To save the image as well #############
for t_step in range(300, 437, 1):
# for t_step in range(t_steps):
    update_plot(t_step)
    plt.savefig(f'../temperoray_figures/Gaussian/TDB/lead_lag/f_{t_step}.png',bbox_inches='tight', dpi=100)
    plt.close() 
print("FINISHED")